# pySCENIC workflow

The present study is based on the 10X scRNA-seq dataset published by the Allen Institute for Brain Science and publicly available at: https://portal.brain-map.org/atlases-and-data/RNA-seq/mouse-whole-cortex-and-hippocampus-10x. The data was then clustered, and cluster names were assigned based on the Allen Institute proposal for cell type nomenclature (https://portal.brain-map.org/explore/classes/nomenclature). The topology of the taxonomy allowed to define the sex of the mouse from which the cells were isolated, the regions of interest, cell classes (glutamatergic, GABAergic or Non-Neuronal) and subclasses. This information was stored in the metadata table. The metadata was used to subset cells of the hippocampus region from the gene expression matrix. We selected for 13 subclasses of hippocampal cells. The hippocampus gene count matrix was pre-processed in R v3.6.1 according to the Seurat v3.1.5 standard pre-processing workflow for quality control, normalization, and analysis of scRNA-seq data (cf. 10XHip2021_Pre.Processing) and transformed in a loom object for further analysis with pySCENIC.

# Description

Here we describe the pySCENIC workflow as applied to the loom object.

# pySCENIC

For a detailed description of the pySCENIC workflow please see here: https://scenic.aertslab.org/

# Data availability

cf. README to download pySCENIC input '10XHip2021.loom'

### 1. Load data, required packages and request CPU/schedule job

In [ ]:
if __name__ == '__main__':
    import argparse
    from pathlib import Path
    parser = argparse.ArgumentParser()
    parser.add_argument("--file",default=None)
    parser.add_argument("--hpc", default=False)
    parser.add_argument("--cpu", default=1)
    parser.add_argument("--threads", default=1)
    filename = parser.parse_args().file
    HPC = parser.parse_args().hpc
    CPU = int(parser.parse_args().cpu)
    Threads = int(parser.parse_args().threads)
    if filename is None:
        filename = input("Datafile name.csv: ")
    while filename.find(".loom") == -1:
        filename = input("File must be .loom, please try again: ")
    
    import anndata as ad
    import pandas as pd
    import loompy as lp
    import numpy as np
    import bokeh
    import os, glob
    import pickle
    
    from distributed import LocalCluster,Client
    
    from arboreto.utils import load_tf_names
    from arboreto.algo import grnboost2
    
    from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
    from pyscenic.utils import modules_from_adjacencies
    from pyscenic.prune import prune2df, df2regulons
    from pyscenic.aucell import aucell
    
    import seaborn as sns
    
    if HPC == False:
        SCENICDB_FOLDER="/path1/pyScenic/" 
        # This folder contains Databases (mm10, mm9 refseq files) and Resources (TF and motifs files)
        INPUT_FOLDER="/path1/1-Input_Scenic" 
        # This folder contains your input loom object
        OUTPUT_FOLDER="/path1/2-Output_Scenic"
        # This folder contains your results/output files
    else: 
        SCENICDB_FOLDER="/path2/pyScenic/"
        INPUT_FOLDER="/path2/1-Input_Scenic"
        OUTPUT_FOLDER="/path2/2-Output_Scenic"

    
    DATA_FOLDER=os.path.join(OUTPUT_FOLDER,(filename.replace(".loom","")+"_Results"))
        
    RESOURCES_FOLDER = os.path.join(SCENICDB_FOLDER, "Resources")
    DATABASE_FOLDER = os.path.join(SCENICDB_FOLDER, "Databases")
    
    if HPC == False:    
        SCHEDULER="address_scheduler1" # Enter the address of scheduler1
    else:
        SCHEDULER="address_scheduler2" # Enter the address of scheduler2
        cluster = LocalCluster(n_workers=CPU,
                             threads_per_worker=Threads,
                             memory_limit='400GB',
                             dashboard_address='address_localhost') # Enter the address of local host
        client = Client(cluster)
        print(cluster.dashboard_link)

### 2. Prepare input and output files

In [ ]:
    #pyScenic Databases
    DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "mm10*.feather")
    
    #pyScenic Resources
    MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")
    MM_TFS_FNAME = os.path.join(RESOURCES_FOLDER, 'mm_tfs.txt')
    
    #Input file
    SC_EXP_FNAME = os.path.join(INPUT_FOLDER, filename)
    
    #Output file
    ADJACENCIES_FNAME = os.path.join(DATA_FOLDER, "adjacencies.tsv")
    MODULES_FNAME = os.path.join(DATA_FOLDER, "modules.p")
    MOTIFS_FNAME = os.path.join(DATA_FOLDER, "motifs.csv")
    REGULONS_FNAME = os.path.join(DATA_FOLDER, "regulons.p")
    REGULONS_CSV_FNAME = os.path.join(DATA_FOLDER,"10XHip2021_GRN.matrix.csv") # Choose name of main output
    META_CSV_FNAME = os.path.join(DATA_FOLDER,"Metadata.csv")
    HEATMAP_FNAME = os.path.join(DATA_FOLDER, "heatmap.png")
    
    N_SAMPLES = 500
    
    #Make output dir
    Path(DATA_FOLDER).mkdir(parents=True, exist_ok=True)

### 3. Get the expression matrix (loom object), the TF list and promoter databases

In [ ]:
    #Get expression matrix as pd.DataFrame
    ds = ad.read_loom(SC_EXP_FNAME,validate=False)
    ds.X = ds.layers['raw']
    ds.var.index = np.array(ds.var.gene_names,dtype=str)
    ds.obs.index = np.array(ds.obs.cell_names,dtype=str)
    pd.DataFrame(ds.obs).to_csv(META_CSV_FNAME)
    ExprMat = pd.DataFrame(ds.to_df())
        
    # Get  TF list
    tf_names = load_tf_names(MM_TFS_FNAME)
    
    # Get promoter region databases
    db_fnames = glob.glob(DATABASES_GLOB)
    def name(fname):
        return os.path.splitext(os.path.basename(fname))[0]
    dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]

### 4. Inference of coexpression modules (GRNBoost2)

In [ ]:
#%%
if __name__ == '__main__':
    ###Inference of coexpression modules, and save results
    adjacencies = grnboost2(expression_data=ExprMat, tf_names=tf_names, verbose=True)
    adjacencies.to_csv(ADJACENCIES_FNAME, index=False, sep='\t')
    
    ###Close clients
    client.close()
    cluster.close()

### 5. Derive potential regulons from these co-expression modules (cisTarget)

#### Regulons are derived from adjacencies based on three methods.
##### The first method to create the TF-modules is to select the best targets for each transcription factor:
- Targets with importance > the 50th percentile.
- Targets with importance > the 75th percentile
- Targets with importance > the 90th percentile.

##### The second method is to select the top targets for a given TF: Top 50 targets (targets with highest weight)

##### The alternative way to create the TF-modules is to select the best regulators for each gene. 
##### Then, these targets can be assigned back to each TF to form the TF-modules. In this way we will create three more gene-sets:
- Targets for which the TF is within its top 5 regulators
- Targets for which the TF is within its top 10 regulators
- Targets for which the TF is within its top 50 regulators

##### A distinction is made between modules which contain targets that are being activated and genes that are being repressed. Relationship between TF and its target, i.e. activator or repressor, is derived using the original expression profiles. The Pearson product-moment correlation coefficient is used to derive this information. In addition, the transcription factor is added to the module and modules that have less than 20 genes are removed.

In [ ]:
    modules = list(modules_from_adjacencies(adjacencies, ExprMat))    
    with open(MODULES_FNAME, 'wb') as f:
        pickle.dump(modules, f)
    #with open(MODULES_FNAME,'rb') as f:
    #    modules = pickle.load(f)
    
    ###Prune modules for targets with cis-Regulatory footprints
    df = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME, num_workers=CPU)
    df.to_csv(MOTIFS_FNAME)
    
    ###Pruned module dataframe can be converted to regulons
    regulons = df2regulons(df)
    with open(REGULONS_FNAME, 'wb') as f:
        pickle.dump(regulons, f)
    #with open(REGULONS_FNAME, 'rb') as f:
    #    regulons = pickle.load(f)

### 6. Characterize cells based on regulon enrichment (AUCell)

In [ ]:
    auc_mtx = aucell(ExprMat, regulons, num_workers=CPU)
    auc_mtx.to_csv(REGULONS_CSV_FNAME)
    sns.clustermap(auc_mtx, figsize=(12,12)).savefig(HEATMAP_FNAME)